In [1]:
import pandas as pd
from src.ipsae import return_ipsae
from Bio import SeqIO
import os
import json

In [2]:
# Load in the data provided by Adaptyv Bio containing the sequences, binding detected, and Kds.
results_df = pd.read_csv("data/result_summary.csv")
# Also load in the AlphaPulldown sequences
fasta_file = "data/all_egfr_seqs.fasta"

In [3]:
# The AlphaPulldown directories have a unique code for each of the sequences rather than the submission name, so we'll map the sequences back to the all_egfr_seqs.fasta file, to get the code, so we can access the pae and pdb files.
# I'll create a dictionary to store these mappings.
seq2id = {}
fasta_data = SeqIO.parse(open(fasta_file), 'fasta')
for fasta in fasta_data:
    name, sequence = fasta.id, str(fasta.seq)
    # here the name is the unique
    seq2id[sequence] = name

In [ ]:
# We are going make a function that takes in the amino acid sequence, and returns the file path to the pae and pdb files from the Alphapulldown directories
def get_pae_file_path(sequence):
    id = seq2id[sequence]
    apd_dir = os.path.join("data/egfr_adaptyv_paper_alphapulldown_paes_pdbs", f"6ARU_1_Chain_A_and_{id}")

    # Check if the directory exists (most of jvogt's sequences didn't seem to fold)
    if not os.path.exists(apd_dir):
        print(f"apd doesnt exist for {id}")
        return -1

    ranking_debug_path = os.path.join(apd_dir, 'ranking_debug.json')

    # Check if ranking_debug.json exists. This is to see which PAE file corresponds to the best model.
    if not os.path.exists(ranking_debug_path):
        print(f"ranking doesnt exist for {id}")
        return -1

    with open(ranking_debug_path, 'r') as file:
        json_data = json.load(file)
        # Get the top-ranked model
        top_model = json_data['order'][0]  

    # Construct the PAE file path
    pae_file_path = os.path.join(apd_dir, f"pae_{top_model}.json")

    # Check if the PAE file exists
    return pae_file_path if os.path.exists(pae_file_path) else -1

def get_pdb_file_path(sequence):
    id = seq2id[sequence]
    apd_dir = os.path.join("data/egfr_adaptyv_paper_alphapulldown_paes_pdbs", f"6ARU_1_Chain_A_and_{id}")
    if not os.path.exists(apd_dir):
        return -1
    
    # I am assuming that ranked_0 is the "best" structure.
    pdb_file_path = os.path.join(apd_dir, "ranked_0.pdb")
    if os.path.exists(pdb_file_path):
        return pdb_file_path 
    else:
        print(f"pdb doesnt exist for {id}") 
        return -1

In [5]:
# These are the default cutoffs.
pae_cutoff = 10
dist_cutoff = 10

# Now we add a new column for ipsae to the dataframe, and save as a csv file for further analysis.
results_df['ipsae'] = results_df['sequence'].apply(lambda seq: return_ipsae(get_pae_file_path(seq), get_pdb_file_path(seq), pae_cutoff, dist_cutoff))
results_df.to_csv("results_with_ipsae.csv")


apd doesnt exist for b1e10681e9b3e5a1b7ec66ca7ba604aa
apd doesnt exist for e0243e84c757f833ec3790cea7ab8b1a
apd doesnt exist for 2e41a6ed50ebc6dd0864b17063f460ba
apd doesnt exist for 1b366cc4a45cefc2e55c908b31f255d4
apd doesnt exist for 22aa87100f1a31cc684c6a805ec90142
apd doesnt exist for 380f3646cfe1f482b8161f4cee30e86a
apd doesnt exist for 6fcb382d2b9294dd5cd2b92427aa4fe2
